# Wavenet Demo
Demo of our efficient generation implementation.

Trains wavenet on a single wav file. Then generates that file, starting from a single sample.

In [ ]:
from time import time
import datetime
import os


from wavenet.utils import make_batch
from wavenet.models import Model, Generator

from IPython.display import Audio
from librosa import display, decompose, output

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
sample_rate = 16000
wav = 'Floating_Points'
path = 'assets/' + wav +'.wav'
duration=6

server_dir = 'datasets/'
local_dir = 'datasets/'

training_done = False

inputs, targets = make_batch(path, sample_rate, duration=duration)

num_time_samples = inputs.shape[1]
num_channels = 1
gpu_fraction = 1.0
num_classes=256
num_blocks=2
num_layers=14
num_hidden=128

model = Model(num_time_samples=num_time_samples,
              num_channels=num_channels,
              gpu_fraction=gpu_fraction,
              num_classes=num_classes, 
              num_blocks=num_blocks, 
              num_layers=num_layers,
              num_hidden=num_hidden)

%matplotlib inline

Audio(inputs.reshape(inputs.shape[1]), rate=sample_rate)

In [ ]:
global_tic = time()
train_step, losses = model.train(inputs, targets)
global_toc = time()

training_time = global_toc-global_tic
print('Training took {} seconds.'.format(training_time))

training_done = True

filename = "lay_{}_qtz_{}_dil_{}_nsp_{}_step_{}".format(num_layers, num_classes, num_hidden, num_time_samples+1,train_step)

plt.plot(losses)
plt.show()

# O(num_sample * num_layers)  => num_classes (quantification) doesn't affect the time complexity
    # num_sample : 3s of training per second of inputs
    #              Sample_rate=44100 => 1_second_of_traning/14000_samples

In [1]:
# To save generated and original samples locally or on the server

for directory in [local_dir] :
    
    new_dir_path = directory + wav 
    
    sample_rate_path = '/' + 'sr_{}'.format(sample_rate)
    
    if not os.path.exists(new_dir_path):
        os.mkdir(new_dir_path)
        print("Directory : " , new_dir_path ,  " created ")
        
    else:    
        print("Directory : " , new_dir_path ,  " already exists")
        
    if not os.path.exists(new_dir_path + sample_rate_path):
        os.mkdir(new_dir_path + sample_rate_path)
        print("Directory : " , new_dir_path + sample_rate_path ,  " created ")
        
    else:    
        print("Directory : " , new_dir_path + sample_rate_path,  " already exists")
        
    if training_done == True :       
        filename = "lay_{}_qtz_{}_dil_{}_nsp_{}_step_{}".format(num_layers, num_classes, num_hidden, num_time_samples+1,train_step)
    
    if training_done == False :        
        filename = "lay_{}_qtz_{}_dil_{}_nsp_{}".format(num_layers, num_classes, num_hidden, num_time_samples+1)

    # Write the original wav file if it doesn't exist
    
    if os.path.isfile(new_dir_path + sample_rate_path + '/original_sr_{}.wav'.format(sample_rate)) == False :        
        output.write_wav(new_dir_path + sample_rate_path + '/original_sr_{}.wav'.format(sample_rate), inputs.reshape(inputs.shape[1]), sample_rate)
        print("original wav written")
        
    else :
        print("original wav already exists")

    #Create and write on a txt the hyperparameters that we used
    if os.path.isfile(new_dir_path + sample_rate_path + '/' + filename + '.txt') == False :
        f = open(new_dir_path + sample_rate_path + '/' + filename + '.txt', "w+")
        print("Text files created")
    
        f.write('The sample rate is : ' + str(sample_rate) + '\n')
        f.write('The duration is : ' + str(duration) + '\n')
        f.write('The number of samples is : ' + str(num_time_samples+1) + '\n')
        f.write('The quantification is : ' + str(num_classes) + '\n')
        f.write('The number of blocks is : ' + str(num_blocks) + '\n')
        f.write('The numbers of layers is : ' + str(num_layers) + '\n')
        f.write('The dilatation is : ' + str(num_hidden) + '\n')
        if training_done == True :
            f.write('The number of train step is : ' + str(train_step) + '\n')
            f.write('The final loss is : ' + str(loss) + '\n')
            f.write('The training time is : ' + str(training_time) + '\n')
        f.write('The generation time is : ' + str(generation_time) + '\n')
    
        print("text file ready")
        f.close()
        
    else:
        print("Text file already exist")
        
    #Save the losses as an image of the plot
    if os.path.isfile(new_dir_path + sample_rate_path + '/' + filename + '-plot' '.png') == False :
        plot.savefig(new_dir_path + sample_rate_path + '/' + filename + '-plot' '.png')
        print('Losses created')
    else :
        print('Losses already exists')



NameError: name 'local_dir' is not defined

In [ ]:
generator = Generator(model)

# Get first sample of input
input_ = inputs[:, 0:1, 0]

global_tic = time()
predictions = generator.run(input_, num_time_samples)
global_toc = time()
generation_time = global_toc-global_tic
print('Generating took {} seconds.'.format(generation_time))

In [ ]:
plt.subplot(1, 1, 1)
display.waveplot(inputs.reshape(inputs.shape[1]), sr=sample_rate)
plt.title('original')

In [ ]:
# The original audio
Audio(inputs.reshape(inputs.shape[1]), rate=sample_rate)

In [ ]:
# The predictions from the first imput

Audio(predictions, rate=sample_rate)

In [ ]:
# Write the generated wav file   
if os.path.isfile(new_dir_path + sample_rate_path + '/' + filename + '.wav') == False :
    output.write_wav(new_dir_path + sample_rate_path + '/' + filename + '.wav', predictions[0], sample_rate)
    print("generated wav written")
else : 
    print("generated wav already exist with those parameters")



In [ ]:
import numpy as np 
global_tic = time()
new_pred = generator.run([[np.random.randn()]], num_time_samples)
global_toc = time()

random_time = global_tic-global_toc
f = open(new_dir_path + sample_rate_path + '/' + filename + '.txt', "w")
f.write('The random generation time is : ' + str(random_time) + '\n')

In [ ]:
%matplotlib inline
Audio(new_pred, rate=sample_rate)

In [ ]:
# Save the random generated file
if os.path.isfile(new_dir_path + sample_rate_path + '/' + filename + '_random' + '.wav') == False :
    output.write_wav(new_dir_path + sample_rate_path + '/' + filename + '_random' + '.wav', new_pred[0], sample_rate)
    print("random generated wav written")
else : 
    print("random generated wav already exist with those parameters")
